In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [21]:
import tensorflow as tf
import tensorflow_hub as hub

# Image shape
IMAGE_SHAPE = (224, 224)

# Load the pre-trained MobileNetV2 model from TensorFlow Hub
mobilenet_layer = hub.KerasLayer(
    "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4",
    input_shape=IMAGE_SHAPE + (3,),  # Add channel dimension
    trainable=False  # Ensure the model is not trainable
)

# Define a preprocessing function
def preprocess_image(image):
    # Normalize image to the [0, 1] range
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize image to the required input shape
    image = tf.image.resize(image, IMAGE_SHAPE)
    return image

# Functional API model
inputs = tf.keras.Input(shape=IMAGE_SHAPE + (3,))
preprocessed_inputs = tf.keras.layers.Lambda(preprocess_image)(inputs)  # Apply preprocessing
outputs = mobilenet_layer(preprocessed_inputs)  # Pass preprocessed inputs to the model
classifier = tf.keras.Model(inputs, outputs)

print("Model with preprocessing loaded successfully!")


ValueError: Exception encountered when calling layer 'keras_layer_6' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_6' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_4>
  • training=None

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [4]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [5]:
list(data_dir.glob('*/*.jpg'))[:5]

[PosixPath('datasets/flower_photos/daisy/5948835387_5a98d39eff_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/14114116486_0bb6649bc1_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/450128527_fd35742d44.jpg'),
 PosixPath('datasets/flower_photos/daisy/8094774544_35465c1c64.jpg'),
 PosixPath('datasets/flower_photos/daisy/8383753520_8391dd80ee_m.jpg')]

In [6]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [7]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [8]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))

In [9]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])
X = np.array(X)
y = np.array(y)

In [10]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [11]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [12]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

In [13]:
x0_resized = cv2.resize(X[0], IMAGE_SHAPE)
x1_resized = cv2.resize(X[1], IMAGE_SHAPE)
x2_resized = cv2.resize(X[2], IMAGE_SHAPE)

In [14]:

predicted = classifier.predict(np.array([x0_resized, x1_resized, x2_resized]))
predicted = np.argmax(predicted, axis=1)
predicted

NameError: name 'classifier' is not defined

In [22]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)
num_of_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7bb1c2493640> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
model.evaluate(X_test_scaled,y_test)